## reading in data and importing things

In [1]:
#importing things
import pandas as pd
import os
import numpy as np
import csv
import pylab as pl
import matplotlib.pyplot as plt
import copy
import scipy
import scipy.stats as stats
from scipy.stats import linregress
from scipy.stats import t
from scipy.stats import chi2
from scipy.stats import sem
from scipy.linalg import hadamard
from scipy.stats import ttest_ind_from_stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import ttest_ind_from_stats
from sklearn.manifold import TSNE
from collections import OrderedDict
import seaborn as sns
from functools import reduce
from scipy.optimize import curve_fit
from operator import itemgetter
import math

In [2]:
#reading in counts
rootdir = '../Raw_data/seq_counts/Expression/6261z'
list_of_dataframes = []
for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if '6261' in file:
                filepath = str(rootdir)+'/'+str(file)
                filename, file_extension = os.path.splitext(file)
                list_of_dataframes.append(pd.read_table(filepath,delimiter='\t\t',names=['variant',filename],index_col=None,dtype={'variant': str,'LogEC50':float}))

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['variant'],
                                            how='outer'), list_of_dataframes)
df_merged.fillna(0, inplace=True)
pd.DataFrame.to_csv(df_merged, 'expression_analysis/6261z_merged_reads.txt', sep=',', na_rep=float(0), index=False)
df_merged

/Users/angelaphillips/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,variant,correct_counts_6261_z_F_7,correct_counts_6261_z_F_6,correct_counts_6261_z_F_4,correct_counts_6261_z_F_5,correct_counts_6261_z_F_1,correct_counts_6261_z_F_2,correct_counts_6261_z_F_3,correct_counts_6261_z_F_8
0,01010100111,532.0,184.0,406.0,435.0,227.0,254.0,183.0,642.0
1,01011111101,399.0,175.0,461.0,597.0,230.0,277.0,229.0,252.0
2,01000100101,374.0,203.0,352.0,448.0,147.0,115.0,115.0,721.0
3,01001100011,358.0,196.0,344.0,447.0,158.0,86.0,181.0,507.0
4,01001111101,340.0,167.0,430.0,436.0,157.0,168.0,210.0,166.0
...,...,...,...,...,...,...,...,...,...
2042,01100110111,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2043,00110110101,0.0,0.0,0.0,0.0,0.0,1.0,1.0,7.0
2044,10100010010,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2045,01101110101,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [3]:
#extracting mean log fluor and stdev log fluor from fcs files
rootdir = '../Raw_data/gate_events/6261z'

concs = ["myc_FITC"]
FITC_min_list=[]
FITC1_bin_avg_list=[]
FITC2_bin_avg_list=[]
FITC3_bin_avg_list=[]
FITC4_bin_avg_list=[]
FITC5_bin_avg_list=[]
FITC6_bin_avg_list=[]
FITC7_bin_avg_list=[]
FITC8_bin_avg_list=[]
FITC1_bin_std_list=[]
FITC2_bin_std_list=[]
FITC3_bin_std_list=[]
FITC4_bin_std_list=[]
FITC5_bin_std_list=[]
FITC6_bin_std_list=[]
FITC7_bin_std_list=[]
FITC8_bin_std_list=[]
for conc in concs:
    #calc min PE fluor to shift all above zero
    FITC_min = 0
    FITC_list=[]
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            filepath = subdir + os.sep + file
            if conc in filepath:
                with open(filepath, encoding="utf8", errors="ignore") as csv_file:
                    csv_reader = csv_file.readlines()
                    for line in csv_reader:
                        FITC_list.append(float(line.split(",")[6]))

    if min(FITC_list) < FITC_min:
        FITC_min = min(FITC_list)
    else:
        FITC_min = FITC_min
    print("FITC_min: "+str(conc)+":"+str(FITC_min))
    FITC_min_list.append(FITC_min)
    
    #bin mean log fluor
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + file
            if conc in filepath:
                if "F1" in filepath:
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC1_bin_avg: "+str(bin_avg))
                        print(conc+"FITC1_bin_std: "+str(bin_std))
                        FITC1_bin_avg_list.append(bin_avg)
                        FITC1_bin_std_list.append(bin_std)
                elif "F2" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC2_bin_avg: "+str(bin_avg))
                        print(conc+"FITC2_bin_std: "+str(bin_std))
                        FITC2_bin_avg_list.append(bin_avg)
                        FITC2_bin_std_list.append(bin_std)
                elif "F3" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC3_bin_avg: "+str(bin_avg))
                        print(conc+"FITC3_bin_std: "+str(bin_std))
                        FITC3_bin_avg_list.append(bin_avg)
                        FITC3_bin_std_list.append(bin_std)
                elif "F4" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC4_bin_avg: "+str(bin_avg))
                        print(conc+"FITC4_bin_std: "+str(bin_std))
                        FITC4_bin_avg_list.append(bin_avg)
                        FITC4_bin_std_list.append(bin_std)
                elif "F5" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC5_bin_avg: "+str(bin_avg))
                        print(conc+"FITC5_bin_std: "+str(bin_std))
                        FITC5_bin_avg_list.append(bin_avg)
                        FITC5_bin_std_list.append(bin_std)
                elif "F6" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC6_bin_avg: "+str(bin_avg))
                        print(conc+"FITC6_bin_std: "+str(bin_std))
                        FITC6_bin_avg_list.append(bin_avg)
                        FITC6_bin_std_list.append(bin_std)
                elif "F7" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC7_bin_avg: "+str(bin_avg))
                        print(conc+"FITC7_bin_std: "+str(bin_std))
                        FITC7_bin_avg_list.append(bin_avg)
                        FITC7_bin_std_list.append(bin_std)
                elif "F8" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC8_bin_avg: "+str(bin_avg))
                        print(conc+"FITC8_bin_std: "+str(bin_std))
                        FITC8_bin_avg_list.append(bin_avg)
                        FITC8_bin_std_list.append(bin_std)
    
with open('expression_analysis/fluor_mean_std_6261z.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerows(zip(concs,FITC_min_list,FITC1_bin_avg_list,FITC2_bin_avg_list,FITC3_bin_avg_list,FITC4_bin_avg_list,FITC5_bin_avg_list,FITC6_bin_avg_list,FITC7_bin_avg_list,FITC8_bin_avg_list,FITC1_bin_std_list,FITC2_bin_std_list,FITC3_bin_std_list,FITC4_bin_std_list,FITC5_bin_std_list,FITC6_bin_std_list,FITC7_bin_std_list,FITC8_bin_std_list))

FITC_min: myc_FITC:-152.32
../Raw_data/gate_events/6261z/Sorted_6261z_myc_FITC_F8.csv
myc_FITCFITC8_bin_avg: 4.207101664204444
myc_FITCFITC8_bin_std: 0.11044681870388541
../Raw_data/gate_events/6261z/Sorted_6261z_myc_FITC_F3.csv
myc_FITCFITC3_bin_avg: 2.8646637035723783
myc_FITCFITC3_bin_std: 0.1527636324168028
../Raw_data/gate_events/6261z/Sorted_6261z_myc_FITC_F2.csv
myc_FITCFITC2_bin_avg: 2.5821975349384525
myc_FITCFITC2_bin_std: 0.046587850806447666
myc_FITCFITC1_bin_avg: 2.3677756201972158
myc_FITCFITC1_bin_std: 0.11259099872055682
../Raw_data/gate_events/6261z/Sorted_6261z_myc_FITC_F5.csv
myc_FITCFITC5_bin_avg: 3.6098060981521813
myc_FITCFITC5_bin_std: 0.053313164576412726
../Raw_data/gate_events/6261z/Sorted_6261z_myc_FITC_F4.csv
myc_FITCFITC4_bin_avg: 3.366980993597842
myc_FITCFITC4_bin_std: 0.09773268318573694
../Raw_data/gate_events/6261z/Sorted_6261z_myc_FITC_F6.csv
myc_FITCFITC6_bin_avg: 3.785612489586806
myc_FITCFITC6_bin_std: 0.050912169048720715
../Raw_data/gate_events/6

In [4]:
#reading in fluor values and cell counts
fluor = pd.DataFrame(pd.read_csv('expression_analysis/fluor_mean_std_6261z.csv',delimiter=',',names=['concen','FITCmin','1','2','3','4','5','6','7','8','1s','2s','3s','4s','5s','6s','7s','8s']))
cellct = pd.DataFrame(pd.read_csv('../Raw_data/cell_counts/Expression/6261z_cellct_exp.csv',delimiter=','))

In [5]:
fluor

,concen,FITCmin,1,2,3,4,5,6,7,8,1s,2s,3s,4s,5s,6s,7s,8s
0,myc_FITC,-152.32,2.367776,2.582198,2.864664,3.366981,3.609806,3.785612,3.964859,4.207102,0.112591,0.046588,0.152764,0.097733,0.053313,0.050912,0.053338,0.110447


## calculate mean bin and read counts

In [6]:
concs = ['_F_']
i = 0 #index for conc
total_ct = []
bin1_cts = []
bin2_cts = []
bin3_cts = []
bin4_cts = []
bin5_cts = []
bin6_cts = []
bin7_cts = []
bin8_cts = []
for conc in concs:
    #first grab the counts and the fluor values
    #total cell counts in bin (a,b,c,d)
    Ca = float(cellct.at[i,'1'])
    Cb = float(cellct.at[i,'2'])
    Cc = float(cellct.at[i,'3'])
    Cd = float(cellct.at[i,'4'])
    Ce = float(cellct.at[i,'5'])
    Cf = float(cellct.at[i,'6'])
    Cg = float(cellct.at[i,'7'])
    Ch = float(cellct.at[i,'8'])
    bin1_ct = df_merged['correct_counts_6261_z'+str(conc)+'1'].sum(axis=0) #sum all counts in a bin
    avg_fluor_1 = fluor.at[i, '1'] #extract mean fluor
    bin2_ct = df_merged['correct_counts_6261_z'+str(conc)+'2'].sum(axis=0) #sum all counts in a bin
    avg_fluor_2 = fluor.at[i, '2'] #extract mean fluor
    bin3_ct = df_merged['correct_counts_6261_z'+str(conc)+'3'].sum(axis=0) #sum all counts in a bin
    avg_fluor_3 = fluor.at[i, '3'] #extract mean fluor
    bin4_ct = df_merged['correct_counts_6261_z'+str(conc)+'4'].sum(axis=0) #sum all counts in a bin
    avg_fluor_4 = fluor.at[i, '4'] #extract mean fluor
    bin5_ct = df_merged['correct_counts_6261_z'+str(conc)+'5'].sum(axis=0) #sum all counts in a bin
    avg_fluor_5 = fluor.at[i, '5'] #extract mean fluor
    bin6_ct = df_merged['correct_counts_6261_z'+str(conc)+'6'].sum(axis=0) #sum all counts in a bin
    avg_fluor_6 = fluor.at[i, '6'] #extract mean fluor
    bin7_ct = df_merged['correct_counts_6261_z'+str(conc)+'7'].sum(axis=0) #sum all counts in a bin
    avg_fluor_7 = fluor.at[i, '7'] #extract mean fluor
    bin8_ct = df_merged['correct_counts_6261_z'+str(conc)+'8'].sum(axis=0) #sum all counts in a bin
    avg_fluor_8 = fluor.at[i, '8'] #extract mean fluor
    total_ct_c = bin1_ct + bin2_ct + bin3_ct + bin4_ct + bin5_ct + bin6_ct + bin7_ct + bin8_ct #sum total count
    total_ct.append(total_ct_c) #append to list
    bin1_cts.append(bin1_ct)
    bin2_cts.append(bin2_ct)
    bin3_cts.append(bin3_ct)
    bin4_cts.append(bin4_ct)
    bin5_cts.append(bin5_ct)
    bin6_cts.append(bin6_ct)
    bin7_cts.append(bin7_ct)
    bin8_cts.append(bin8_ct)
    i = i + 1
    #now calc the var frequency
    df_merged['mean_bin1_%s'%conc] = (100*Ca*df_merged['correct_counts_6261_z'+str(conc)+'1']/bin1_ct)
    df_merged['mean_bin2_%s'%conc] = (100*Cb*df_merged['correct_counts_6261_z'+str(conc)+'2']/bin2_ct)
    df_merged['mean_bin3_%s'%conc] = (100*Cc*df_merged['correct_counts_6261_z'+str(conc)+'3']/bin3_ct)
    df_merged['mean_bin4_%s'%conc] = (100*Cd*df_merged['correct_counts_6261_z'+str(conc)+'4']/bin4_ct)
    df_merged['mean_bin5_%s'%conc] = (100*Ce*df_merged['correct_counts_6261_z'+str(conc)+'5']/bin5_ct)
    df_merged['mean_bin6_%s'%conc] = (100*Cf*df_merged['correct_counts_6261_z'+str(conc)+'6']/bin6_ct)
    df_merged['mean_bin7_%s'%conc] = (100*Cg*df_merged['correct_counts_6261_z'+str(conc)+'7']/bin7_ct)
    df_merged['mean_bin8_%s'%conc] = (100*Ch*df_merged['correct_counts_6261_z'+str(conc)+'8']/bin8_ct)
    #now calc the meanbin
    df_merged['meanbin_%s'%conc] = (df_merged['mean_bin1_%s'%conc] * float(avg_fluor_1) + df_merged['mean_bin2_%s'%conc] * float(avg_fluor_2) + df_merged['mean_bin3_%s'%conc] * float(avg_fluor_3) + df_merged['mean_bin4_%s'%conc] * float(avg_fluor_4) + df_merged['mean_bin5_%s'%conc] * float(avg_fluor_5) + df_merged['mean_bin6_%s'%conc] * float(avg_fluor_6) + df_merged['mean_bin7_%s'%conc] * float(avg_fluor_7) + df_merged['mean_bin8_%s'%conc] * float(avg_fluor_8))/(df_merged['mean_bin1_%s'%conc]+df_merged['mean_bin2_%s'%conc]+df_merged['mean_bin3_%s'%conc]+df_merged['mean_bin4_%s'%conc]+df_merged['mean_bin5_%s'%conc]+df_merged['mean_bin6_%s'%conc]+df_merged['mean_bin7_%s'%conc]+df_merged['mean_bin8_%s'%conc])
df_totals = pd.DataFrame(list(zip(concs, total_ct, bin1_cts, bin2_cts, bin3_cts, bin4_cts, bin5_cts, bin6_cts, bin7_cts, bin8_cts)), columns =['concen', 'total_ct', '1','2','3','4','5', '6','7','8'])
pd.DataFrame.to_csv(df_merged,'expression_analysis/expression_meanbins_6261z.csv')

In [34]:
##merging all expression data
a = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_6261a.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
a.rename(columns={'meanbin__F_':'meanbin_a'},inplace=True)
b = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_6261b.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
b.rename(columns={'meanbin__F_':'meanbin_b'},inplace=True)
c = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_6261c.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
c.rename(columns={'meanbin__F_':'meanbin_c'},inplace=True)
x = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_6261x.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
x.rename(columns={'meanbin__F_':'meanbin_x'},inplace=True)
y = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_6261y.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
y.rename(columns={'meanbin__F_':'meanbin_y'},inplace=True)
z = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_6261z.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
z.rename(columns={'meanbin__F_':'meanbin_z'},inplace=True)

In [36]:
CR6261 = pd.DataFrame()
CR6261 = pd.merge(a, b, on='variant',how='outer')
CR6261 = pd.merge(CR6261, c, on='variant',how='outer')
CR6261 = pd.merge(CR6261, x, on='variant',how='outer')
CR6261 = pd.merge(CR6261, y, on='variant',how='outer')
CR6261 = pd.merge(CR6261, z, on='variant',how='outer')
CR6261['exp_mean']=CR6261.mean(axis=1)
CR6261['exp_sem']=CR6261.sem(axis=1)
pd.DataFrame.to_csv(CR6261,'expression_analysis/exp_meanbin_merged.csv')